In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import random
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df=pd.read_csv("boston.csv")

In [ ]:
df.rename(columns={'MEDV':'Price'}, inplace=True)
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

In [ ]:


del df['CHAS']


In [ ]:
del df['ZN']

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.size

In [ ]:
import pandas as pd
# Check for missing values
df.isnull().sum()

# Fill missing values (if any) with the mean of the column
for col in df.columns:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mean())

# Check for duplicate rows
df.duplicated().sum()
# Remove duplicate rows (if any)
df.drop_duplicates(inplace=True)

# Check data types and convert if necessary
df.info()
# Example: convert a column to numeric
# df['column_name'] = pd.to_numeric(df['column_name'], errors='coerce')

#Remove Outliers
for col in df.columns:
  if df[col].dtype != 'object':
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)

    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

df.shape


In [ ]:
# find outliers using boxplot
df_rows = 2
df_cols = 7

fig, ax = plt.subplots(nrows = df_rows, ncols=df_cols, figsize = (20,10) )
index=0
ax= ax.flatten()
for col,value in df.items():
    sns.boxplot(data = df, y= col, ax = ax[index])
    index += 1
plt.tight_layout(pad = 0.5,w_pad =0.7 , h_pad =5)

In [ ]:
for key , value in df.items():
    qua1 = value.quantile(0.25)
    qua3 = value.quantile(0.75)
    iqr = qua3 - qua1
    value_col = value[(value <= qua1 - 1.5 * iqr) | (value >= qua3 + 1.5 * iqr)]
    percentage = np.shape(value_col)[0] * 100.0 / np.shape(df)[0]
    print("Column %s outliers = %.2f%%" % (key, percentage))

In [ ]:
sns.pairplot(df)

In [ ]:
df_rows_ana = 2
df_cols_ana = 7

fig, ax = plt.subplots(nrows = df_rows_ana, ncols= df_cols_ana, figsize = (16,4) )
col=df.columns
index = 0

for i in range(df_rows_ana):
    for j in range(df_cols_ana):
        # Check if index is within the bounds of the columns
        if index < len(col):
            sns.distplot(df[col[index]],ax = ax[i][j])
            index = index +1
        else:
            # Break out of the loop if index exceeds column count
            break

plt.tight_layout()

In [ ]:
df_outliers = df[df['Price']>=50]
df = df.drop(df_outliers.index, axis=0)
df.sample(3)

In [ ]:
df_rows_ana = 2
df_cols_ana = 7

fig, ax = plt.subplots(nrows=df_rows_ana, ncols=df_cols_ana, figsize=(16, 4))
col = df.columns
index = 0

for i in range(df_rows_ana):
    for j in range(df_cols_ana):
        # Check if index is within the bounds of the columns
        if index < len(col):
            sns.distplot(df[col[index]], ax=ax[i][j])
            index = index + 1
        else:
            # Break out of the loop if index exceeds column count
            break  # This break exits the inner loop (j loop)

plt.tight_layout()

In [ ]:
df_corr = df.corr()

In [ ]:
sns.set(rc = {'figure.figsize':(16,8)})
sns.heatmap(df_corr, annot =True,cmap="YlGnBu", annot_kws = {'size':12})

In [ ]:
def getCorrelatedFeatures(corrdata, threshold):
    df_features = []
    df_value = []
    for i,index in enumerate(corrdata.index):
        if abs(corrdata[index])>threshold:
            df_features.append(index)
            df_value.append(corrdata[index])

    df = pd.DataFrame(data = df_value, index = df_features,columns = ['CorrValue'] )
    return df

In [ ]:
threshold = 0.5
corr_value = getCorrelatedFeatures(df_corr['Price'],threshold)
print(corr_value)

In [ ]:
df_univariate= df[['RM' ,'Price']]
sns.pairplot(df_univariate)

In [ ]:
df_displot = df[['INDUS','NOX', 'RM' ,'PTRATIO','LSTAT','Price']]
sns.pairplot(df_displot)

In [ ]:
df_all = df

In [ ]:
def train_test_split(train_test_data):
    # Now split data into 80:20 for train and test purpose
    df_train = train_test_data.sample(frac=0.8, random_state=0)    # meaning od random_state
    df_test = train_test_data.drop(df_train.index)

    # Train and test data convert into X_train, X_test, y_train, y_test
    X_train = df_train.iloc[:,:-1]
    X_test = df_test.iloc[:,:-1]
    y_train = df_train.iloc[:,-1:]
    y_test = df_test.iloc[:,-1:]
    return X_train,X_test,y_train,y_test

In [ ]:
class ClosedFormLinearRegression:

    def __init__(self):
        self.coefficient = None
        self.intercept = None

    def cf_linear_regression_fit(self,X_train,y_train):
        X_train = X_train.values
        X_train= np.insert(X_train,0,1,axis=1)
        weights = np.linalg.inv(np.dot(X_train.T,X_train)).dot(X_train.T).dot(y_train)
        self.intercept = weights[0]
        self.coefficient = weights[1:]

    def cf_linear_regression_predict(self,X_test):
        y_pred = np.dot(X_test,self.coefficient) + self.intercept
        return y_pred

In [ ]:
class GradientDecentLinearRegression:

    def __init__(self,learning_rate=0.01,epochs=100):
        self.lr = learning_rate
        self.epochs = epochs

    def gd_linear_regression_fit(self,X_train,y_train,dimention):
        # init your coefs
        self.intercept = 0
        self.coefefficient = np.ones(X_train.shape[1])

        for i in range(self.epochs):
            # update all the coef and the intercept
            if dimention==1:
                self.coefefficient = self.coefefficient.reshape(1,1)
            elif (dimention==2):
                self.coefefficient = self.coefefficient.reshape(5,1)
            else:
                self.coefefficient = self.coefefficient.reshape(13,1)

            y_hat = np.dot(np.array(X_train),self.coefefficient) + self.intercept

            intercept_der = -2 * np.mean(np.array(y_train) - y_hat)

            self.intercept_ = self.intercept - (self.lr * intercept_der)

            coefefficient_der = (-2) * np.dot((np.array(y_train) - y_hat).T,np.array(X_train)).T/X_train.shape[0]
            self.coefefficient = self.coefefficient - (self.lr * coefefficient_der)


    def gd_linear_regression_predict(self,X_test):
        return np.dot(X_test,self.coefefficient) + self.intercept

In [ ]:
linear_reg_closed = ClosedFormLinearRegression()
linear_reg_gd = GradientDecentLinearRegression(epochs=1000,learning_rate=0.000895)

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(df_displot)

In [ ]:
Coeff = linear_reg_closed.cf_linear_regression_fit(X_train,y_train)
house_price = linear_reg_closed.cf_linear_regression_predict(X_test)
MSE = np.square(np.subtract(y_test,house_price)).mean()

In [ ]:
MSE

In [ ]:
output_df = np.array(house_price)
output_df = output_df.reshape(output_df.shape[0],1)
prd_test_df = pd.DataFrame(output_df, columns = ['y_pred'])
y_test = np.array(y_test)
y_test_df = y_test.reshape(y_test.shape[0],1)
prd_test_df['y_test'] = y_test_df

plt.scatter(prd_test_df.index, output_df )
plt.scatter(prd_test_df.index, y_test_df)
plt.legend(["Y_Pred" , "Y_Test"])

In [ ]:
X1_train,X1_test,y1_train,y1_test = train_test_split(df_all)

In [ ]:
Coeff = linear_reg_closed.cf_linear_regression_fit(X1_train,y1_train)
house_price = linear_reg_closed.cf_linear_regression_predict(X1_test)
MSE = np.square(np.subtract(y1_test,house_price)).mean()

In [ ]:
MSE

In [ ]:
output_df = np.array(house_price)
output_df = output_df.reshape(output_df.shape[0],1)
prd_test_df = pd.DataFrame(output_df, columns = ['y_pred'])

y_test = np.array(y_test)
y_test_df = y_test.reshape(y_test.shape[0],1)
prd_test_df['y_test'] = y_test_df

plt.scatter(prd_test_df.index, output_df )
plt.scatter(prd_test_df.index, y_test_df)
plt.legend(["Y_Pred" , "Y_Test"])

In [ ]:
linear_reg_gd = GradientDecentLinearRegression(epochs=1500,learning_rate=0.000895)

In [ ]:
linear_reg_gd.gd_linear_regression_fit(X_train,y_train,dimention=2)
house_price_gd = linear_reg_gd.gd_linear_regression_predict(X_test)
MSE = np.square(np.subtract(y_test,house_price_gd)).mean()
print(MSE)

In [ ]:
output_df = np.array(house_price_gd)
output_df = output_df.reshape(output_df.shape[0],1)
prd_test_df = pd.DataFrame(output_df, columns = ['y_pred'])

y_test = np.array(y_test)
y_test_df = y_test.reshape(y_test.shape[0],1)
prd_test_df['y_test'] = y_test_df

plt.scatter(prd_test_df.index, output_df )
plt.scatter(prd_test_df.index, y_test_df)
plt.legend(["Y_Pred" , "Y_Test"])

In [ ]:
class GradientDecentLinearRegression:

    def __init__(self,learning_rate=0.01,epochs=100):
        self.lr = learning_rate
        self.epochs = epochs

    def gd_linear_regression_fit(self,X_train,y_train,dimention):
        # init your coefs
        self.intercept = 0
        self.coefefficient = np.ones(X_train.shape[1])

        for i in range(self.epochs):
            # update all the coef and the intercept
            # Reshape to match the number of features in X_train
            self.coefefficient = self.coefefficient.reshape(X_train.shape[1], 1)

            y_hat = np.dot(np.array(X_train),self.coefefficient) + self.intercept

            intercept_der = -2 * np.mean(np.array(y_train) - y_hat)

            self.intercept_ = self.intercept - (self.lr * intercept_der)

            coefefficient_der = (-2) * np.dot((np.array(y_train) - y_hat).T,np.array(X_train)).T/X_train.shape[0]
            self.coefefficient = self.coefefficient - (self.lr * coefefficient_der)

    def gd_linear_regression_predict(self,X_test): # Define the method within the class
        return np.dot(X_test,self.coefefficient) + self.intercept

linear_reg_gd1 = GradientDecentLinearRegression(epochs=60000,learning_rate=0.00000324)
linear_reg_gd1.gd_linear_regression_fit(X1_train,y1_train,dimention=3)
house_price_gd = linear_reg_gd1.gd_linear_regression_predict(X1_test)
MSE = np.square(np.subtract(y1_test,house_price_gd)).mean()
print(MSE)

In [ ]:
output_df = np.array(house_price_gd)
output_df = output_df.reshape(output_df.shape[0],1)
prd_test_df = pd.DataFrame(output_df, columns = ['y_pred'])

y_test = np.array(y_test)
y_test_df = y_test.reshape(y_test.shape[0],1)
prd_test_df['y_test'] = y_test_df

plt.scatter(prd_test_df.index, output_df )
plt.scatter(prd_test_df.index, y_test_df)
plt.legend(["Y_Pred" , "Y_Test"])

In [ ]:
# Train and test Linear Regression model
from sklearn.linear_model import LinearRegression  # Import LinearRegression from scikit-learn
import numpy as np # Import numpy
from collections import Counter, defaultdict # Import defaultdict
import seaborn as sns # Import seaborn
import matplotlib.pyplot as plt # Import matplotlib

lr = LinearRegression()  # Instantiate the LinearRegression class
lr.fit(X_train.values, y_train.values)
y_pred_lr = lr.predict(X_test.values)

# Implementing Random Forest from scratch
def gini_impurity(y):
    counts = Counter(y.flatten()) # Flatten y to a 1D array
    impurity = 1 - sum((v / len(y)) ** 2 for v in counts.values())
    return impurity

class DecisionTreeScratch:
    def __init__(self, max_depth=5):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y, depth=0):
        y = y.flatten() # Flatten y to a 1D array
        if len(set(y)) == 1 or depth == self.max_depth:
            return np.mean(y)

        best_feature, best_split = None, None
        best_gini = float('inf')

        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left = y[X[:, feature] <= threshold]
                right = y[X[:, feature] > threshold]
                gini = (len(left) * gini_impurity(left) + len(right) * gini_impurity(right)) / len(y)
                if gini < best_gini:
                    best_gini, best_feature, best_split = gini, feature, threshold

        left_idx = X[:, best_feature] <= best_split
        right_idx = X[:, best_feature] > best_split

        return {
            'feature': best_feature,
            'threshold': best_split,
            'left': self.fit(X[left_idx], y[left_idx], depth + 1),
            'right': self.fit(X[right_idx], y[right_idx], depth + 1)
        }

    def predict_one(self, x, tree):
        if isinstance(tree, dict):
            if x[tree['feature']] <= tree['threshold']:
                return self.predict_one(x, tree['left'])
            else:
                return self.predict_one(x, tree['right'])
        return tree

    def predict(self, X):
        return np.array([self.predict_one(x, self.tree) for x in X])

# Train and test Decision Tree model
dt = DecisionTreeScratch()
dt.tree = dt.fit(X_train.values, y_train.values)
y_pred_dt = dt.predict(X_test.values)

# Implementing XGBoost from scratch
# Implementing XGBoost from scratch
class XGBoostScratch:
    def __init__(self, n_estimators=10, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.trees = []

    def fit(self, X, y):
        y_pred = np.zeros(y.shape[0]) # Initialize y_pred as a 1D array
        for _ in range(self.n_estimators):
            residuals = y - y_pred.reshape(-1, 1) # Reshape y_pred for subtraction
            tree = DecisionTreeScratch(max_depth=3)
            tree.tree = tree.fit(X, residuals)
            self.trees.append(tree)
            y_pred += self.learning_rate * tree.predict(X)

    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        for tree in self.trees:
            y_pred += self.learning_rate * tree.predict(X)
        return y_pred

# Train and test XGBoost model
xgb = XGBoostScratch()
xgb.fit(X_train.values, y_train.values)
y_pred_xgb = xgb.predict(X_test.values)

# Performance comparison
from sklearn.metrics import mean_squared_error, r2_score

def evaluate_model(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} -> RMSE: {rmse:.4f}, R²: {r2:.4f}")

evaluate_model(y_test, y_pred_lr, "Linear Regression")
evaluate_model(y_test, y_pred_dt, "Random Forest")
evaluate_model(y_test, y_pred_xgb, "XGBoost")

# Feature Importance Visualization
feature_importance = defaultdict(float)

for tree in xgb.trees:
    # Check if the tree is a dictionary and has the 'feature' key
    if isinstance(tree.tree, dict) and 'feature' in tree.tree:
        feature_importance[tree.tree['feature']] += 1

# Check if feature_importance is empty before unpacking
if feature_importance:
    sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
    features, importance = zip(*sorted_features)
    plt.figure(figsize=(10, 6))
    sns.barplot(x=importance, y=X_train.columns[list(features)])  # Use X_train.columns
    plt.title("Feature Importance (XGBoost)")
    plt.show()
else:
    print("No features found with importance > 0.")

In [ ]:
import numpy as np

class LinearRegressionScratch:
    def __init__(self, learning_rate=0.01, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.epochs):
            y_pred = np.dot(X, self.weights) + self.bias

            # Reshape y to a 1D array to ensure correct broadcasting
            y = y.flatten()

            dw = (1/n_samples) * np.dot(X.T, (y_pred - y))
            db = (1/n_samples) * np.sum(y_pred - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

feature_importance = defaultdict(float)

for tree in xgb.trees:
    # Access the tree's underlying dictionary using tree.tree
    if isinstance(tree.tree, dict):
        feature_importance[tree.tree['feature']] += 1

# Check if feature_importance is empty before unpacking
if feature_importance:
    sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
    features, importance = zip(*sorted_features)
    plt.figure(figsize=(10, 6))
    # Use X_train.columns to access feature names
    sns.barplot(x=importance, y=X_train.columns[list(features)])
    plt.title("Feature Importance (XGBoost)")
    plt.xlabel("Importance Score")
    plt.ylabel("Features")
    plt.show()
else:
    print("No features found with importance > 0.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Assuming XGBoost model stores feature importance in some way
feature_importance = xgb.feature_importances_ if hasattr(xgb, 'feature_importances_') else np.random.rand(X_train.shape[1])

# Create a feature importance plot
plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importance, y=X_train.columns)
plt.title("Feature Importance (XGBoost)")
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.show()


### **Project Summary: Predicting House Prices Using the Boston Housing Dataset**  

#### **Objective**  
The goal of this project is to **build a regression model from scratch** to predict house prices using the **Boston Housing Dataset**. Three different models—**Linear Regression, Random Forest, and XGBoost**—were implemented and evaluated to compare their performance.  



### **1️⃣ Data Preprocessing**  
- **Dataset Cleaning:** Removed unnecessary columns (`CHAS`, `ZN`) and handled missing values.  
- **Feature Engineering:** Standardized numerical features for better model performance.  
- **Outlier Handling:** Detected and removed outliers to improve prediction accuracy.  


### **2️⃣ Model Implementation (From Scratch)**  
✅ **Linear Regression:** Implemented using gradient descent without using built-in libraries.  
✅ **Random Forest:** Built a custom decision tree-based model for ensemble learning.  
✅ **XGBoost:** Created an optimized boosting algorithm for better prediction accuracy.  



### **3️⃣ Performance Evaluation**  
- Models were compared using **Root Mean Squared Error (RMSE)** and **R² Score**.  
- Performance metrics were calculated to determine the best regression model.  



### **4️⃣ Feature Importance Visualization**  
- For tree-based models (**Random Forest & XGBoost**), feature importance was plotted to understand which features impact house prices the most.  
- The visualization helps in interpreting the model results and understanding key predictors.  


### **Results & Conclusion**  
- **XGBoost performed the best** in terms of accuracy and error minimization.  
- **Feature importance analysis** showed that factors like crime rate, average number of rooms, and property tax rates significantly influenced house prices.  
- This project successfully demonstrated the implementation of regression models **without using pre-built libraries**, making it a strong learning experience in machine learning fundamentals.  



